# 将netcdf数据导出到excel方便后续计算

In [4]:
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

import sys
sys.path.append('../../../src/')
from namelist import *
from mask import polygon_to_mask

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [5]:
years = np.array([2019,2021,2022])

summer = {}
autumn = {}
sensit = {}

for year in tqdm(years):
    # Simulation
    ds_summer = xr.open_dataset(processed_dir + f"Jul_{year}/Jul_{year}_chem.nc")
    ds_autumn = xr.open_dataset(processed_dir + f"Sep_{year}/Sep_{year}_chem.nc")
    ds_sensit = xr.open_dataset(processed_dir + f"Seasonally_{year}/Sep_{year}_chem.nc")
    
    summer_data = ds_summer['O3'][:,0,:,:]
    autumn_data = ds_autumn['O3'][:,0,:,:]
    sensit_data = ds_sensit['O3'][:,0,:,:]
    
    lon = summer_data.longitude
    lat = summer_data.latitude
    shp = gpd.read_file(shp_files['PRD_adm'])
    mask = polygon_to_mask(shp.geometry[0], lon, lat)
    mask_da = xr.DataArray(mask, dims=('y', 'x'))
    masked_summer = summer_data.where(mask_da)
    masked_autumn = autumn_data.where(mask_da)
    masked_sensit = sensit_data.where(mask_da)
    
    sim_summer = masked_summer.mean(dim=('x','y'),skipna=True)
    sim_autumn = masked_autumn.mean(dim=('x','y'),skipna=True)
    sim_sensit = masked_sensit.mean(dim=('x','y'),skipna=True)
    
    summer[f'sim_{year}'] = sim_summer.values
    autumn[f'sim_{year}'] = sim_autumn.values
    sensit[f'sim_{year}'] = sim_sensit.values
    
    # Observation
    obspath = get_obspath('Jul')
    df_summer = pd.read_excel(obspath + f'site_O3_{year}.xlsx',index_col=0)
    obspath = get_obspath('Sep')
    df_autumn = pd.read_excel(obspath + f'site_O3_{year}.xlsx',index_col=0)
    obs_summer = df_summer.mean(axis=1,skipna=True)
    obs_autumn = df_autumn.mean(axis=1,skipna=True)
    
    summer[f'obs_{year}'] = obs_summer.values
    autumn[f'obs_{year}'] = obs_autumn.values

100%|██████████| 3/3 [00:24<00:00,  8.04s/it]


In [21]:
df_summer = pd.DataFrame(
    summer,
    columns=['sim_2019', 'obs_2019', 'sim_2021', 'obs_2021', 'sim_2022', 'obs_2022'],
    index = pd.date_range('2023-07-01T00','2023-07-31T23',freq='H'),
)
df_autumn = pd.DataFrame(
    autumn,
    columns=['sim_2019', 'obs_2019', 'sim_2021', 'obs_2021', 'sim_2022', 'obs_2022'],
    index = pd.date_range('2023-09-01T00','2023-09-30T23',freq='H'),
)
df_sensit = pd.DataFrame(
    sensit,
    columns=['sim_2019', 'sim_2021', 'sim_2022'],
    index = pd.date_range('2023-09-01T00','2023-09-30T23',freq='H'),
)

In [23]:
outpath = 'D:/data/Graduation/Contribution/contribution_hours_Seasonally/'
df_summer.to_excel(outpath + 'summer.xlsx',index=True)
df_autumn.to_excel(outpath + 'autumn.xlsx',index=True)
df_sensit.to_excel(outpath + 'sensit.xlsx',index=True)